In [2]:
import findspark
findspark.init()

import pyspark

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [3]:
 
spark = SparkSession.builder.appName("RFM_Analysis_with_PySpark").getOrCreate()

In [7]:
# data = spark.read.format("csv").option("header", "true").load("data.csv")
df = spark.read.format("csv").option("header", "true").load("V:\data_Science\CustomerSegmentation\data.csv\data.csv")


In [10]:
df = df.withColumn("tmp",split(col("InvoiceDate"),"\s+")).withColumn("date_new",col("tmp").getItem(0))    
df = df.withColumn("DateInvoice", regexp_replace(col("date_new"), "/", "-"))                                 
df = df.withColumn("Date", to_date(col("DateInvoice"),"M-d-yyyy"))                                         
df = df.withColumn("Quantity", df["Quantity"].cast(IntegerType()))                                        
df = df.withColumn("UnitPrice", df["UnitPrice"].cast(DoubleType()))                                          
df = df.select('InvoiceNo','StockCode','Description','Quantity','UnitPrice','CustomerID','Country','Date')  
df.show()

+---------+---------+--------------------+--------+---------+----------+--------------+----------+
|InvoiceNo|StockCode|         Description|Quantity|UnitPrice|CustomerID|       Country|      Date|
+---------+---------+--------------------+--------+---------+----------+--------------+----------+
|   536365|   85123A|WHITE HANGING HEA...|       6|     2.55|     17850|United Kingdom|2010-12-01|
|   536365|    71053| WHITE METAL LANTERN|       6|     3.39|     17850|United Kingdom|2010-12-01|
|   536365|   84406B|CREAM CUPID HEART...|       8|     2.75|     17850|United Kingdom|2010-12-01|
|   536365|   84029G|KNITTED UNION FLA...|       6|     3.39|     17850|United Kingdom|2010-12-01|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|     3.39|     17850|United Kingdom|2010-12-01|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|     7.65|     17850|United Kingdom|2010-12-01|
|   536365|    21730|GLASS STAR FROSTE...|       6|     4.25|     17850|United Kingdom|2010-12-01|
|   536366

In [11]:
df = df.withColumn("TotalPrice", round(df["UnitPrice"] * df["Quantity"], 2))
df = df.withColumn("RecencyDays", expr("datediff('2011-12-31', Date)"))
df.show()

+---------+---------+--------------------+--------+---------+----------+--------------+----------+----------+-----------+
|InvoiceNo|StockCode|         Description|Quantity|UnitPrice|CustomerID|       Country|      Date|TotalPrice|RecencyDays|
+---------+---------+--------------------+--------+---------+----------+--------------+----------+----------+-----------+
|   536365|   85123A|WHITE HANGING HEA...|       6|     2.55|     17850|United Kingdom|2010-12-01|      15.3|        395|
|   536365|    71053| WHITE METAL LANTERN|       6|     3.39|     17850|United Kingdom|2010-12-01|     20.34|        395|
|   536365|   84406B|CREAM CUPID HEART...|       8|     2.75|     17850|United Kingdom|2010-12-01|      22.0|        395|
|   536365|   84029G|KNITTED UNION FLA...|       6|     3.39|     17850|United Kingdom|2010-12-01|     20.34|        395|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|     3.39|     17850|United Kingdom|2010-12-01|     20.34|        395|
|   536365|    22752|SET

In [12]:
rfm_table = df.groupBy('CustomerId')\
              .agg(min('RecencyDays').alias('Recency'), \
                   count('InvoiceNo').alias('Frequency'), \
                 sum('TotalPrice').alias('Monetary'))
rfm_table.show(5)

+----------+-------+---------+------------------+
|CustomerId|Recency|Frequency|          Monetary|
+----------+-------+---------+------------------+
|     16250|    283|       24|            389.44|
|     15574|    199|      168| 702.2500000000002|
|     15555|     34|      925| 4758.200000000001|
|     15271|     29|      275|2485.8199999999997|
|     17714|    342|       10|             153.0|
+----------+-------+---------+------------------+
only showing top 5 rows



In [15]:
import pandas as pd

In [16]:
# rfm_table.show(5)
rfm_table.toPandas().to_csv('improved_data.csv')